In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import shutil

import time

In [ ]:
root_dir='.'

frame_dir=os.path.join(root_dir,'AVA','frames')
if os.path.isdir(frame_dir):
    shutil.rmtree(frame_dir)
os.makedirs(frame_dir,exist_ok=True)

frame_list_dir=os.path.join(root_dir,'AVA','frame_lists')
if os.path.isdir(frame_list_dir):
    shutil.rmtree(frame_list_dir)
os.makedirs(frame_list_dir,exist_ok=True)

video_dir=os.path.join(root_dir,'AVA','videos')
if os.path.isdir(video_dir):
    shutil.rmtree(video_dir)
os.makedirs(video_dir,exist_ok=True)

video_dir=os.path.join(root_dir,'AVA','annotations')
if os.path.isdir(video_dir):
    shutil.rmtree(video_dir)
os.makedirs(video_dir,exist_ok=True)

test_videos=[vi for vi in os.listdir('test_video') if '.mp4' in vi]

In [ ]:
csvs={'train':[],'val':[]}
detect_csvs={'train':[],'val':[]}
bad_csvs={'train':[],'val':[]}
frame_lists={'train':['original_vido_id video_id frame_id path labels'],
             'val':['original_vido_id video_id frame_id path labels']}

start_time=time.time()

for vi in test_videos:

    if vi=='PIGS021219a_000545.mp4':
        kind='val'
    else:
        kind='train'
    
    track_info =np.load(os.path.join(root_dir,'test_video',vi.split('.')[0],'tracklets.npy'),allow_pickle=True) # see gen_YOLO_data.ipynb for explanation
    frame_rate=track_info[0]
    resolution=track_info[2]

    time_now=time.time()
    print('='*40)
    print('Working on',vi)
    print('time now =',time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
    print('time_taken =',round(time_now-start_time,2),'s')
    start_time=time_now

    track=np.load(os.path.join(root_dir,'test_video',vi.split('.')[0],'behaviour_15.npy'),allow_pickle=True)

    # prepare the directory to store the frame data
    frame_dir=os.path.join(root_dir,'AVA','frames',vi.split('.')[0])
    if not os.path.isdir(frame_dir):
        os.mkdir(frame_dir)

    pig_video_path=os.path.join(root_dir,'test_video',vi)
    cap = cv2.VideoCapture(pig_video_path)

    frame_cnt=0
    while (True):
        ret, frame = cap.read() 
        if ret:

            frame_dst_file=os.path.join(root_dir,'AVA','frames',vi.split('.')[0],vi.split('.')[0]+'_'+str(frame_cnt)+'.jpg')
            cv2.imwrite(frame_dst_file, frame)

            content=vi.split('.')[0]+' ' # original_vido_id
            content+= str(0)+' '#str(video_id)+' ' # video_id
            content+= str(frame_cnt)+' ' # frame_id
            content+= frame_dst_file+' ' # path
            content+= '""' # labels
            frame_lists[kind].append(content)

            frame_cnt+=1
            
        else: 
            break

    cv2.destroyAllWindows() 
    cap.release() 

    # extract the box and action label info
    #
    all_boxes=dict() # all_boxes[frame_ind][pig_ind]=csv
    detect_boxes=dict()

    seconds_cnt=0
    for i in range(len(track)): # loop over pig index
        for m in range(len(track[i])): # loop over the frame

            # if m in bad_frames:
            #     continue

            if m==0 or not m%30==29:
                continue

            seconds_cnt+=1

            csv=''
            detect_csv=''

            # (1st col) videonames
            csv+=vi.split('.')[0]+','#PIGS_date_dir+'_'+time_index+',' 
            detect_csv+=vi.split('.')[0]+','#PIGS_date_dir+'_'+time_index+',' 

            # (2nd col) start seconds
            csv+=str(seconds_cnt)+','
            detect_csv+=str(seconds_cnt)+','

            # (3-6th col) coordinates of the vertices of the detection boxes
            x0,x1=track[i][m][1]/resolution[0],track[i][m][3]/resolution[0]
            y0,y1=track[i][m][2]/resolution[1],track[i][m][4]/resolution[1]
            csv+=str(x0)+','+str(y1)+','+str(x1)+','+str(y0)+',' #lower left and upper right
            detect_csv+=str(x0)+','+str(y1)+','+str(x1)+','+str(y0)+',' #lower left and upper right

            # (7th col) behviour id
            action_code=track[i][m][13]
            if action_code==101:
                action_code=7
            elif action_code==102:
                action_code=8
            action_code+=1 # to match those in the .pbtxt file # Oli 
            csv+=str(int(action_code))+','
            detect_csv+=str(int(action_code))+','

            # (8th col) identity ID
            csv+=str(i)

            # (8th col) accuracy for the detection box
            detect_csv+=str(0.99)

            # store the csv info
            if seconds_cnt not in all_boxes:
                all_boxes[seconds_cnt]={i:csv}
                detect_boxes[seconds_cnt]={i:detect_csv}
            else:
                all_boxes[seconds_cnt][i]=csv # all_boxes[frame_ind][pig_ind]
                detect_boxes[seconds_cnt][i]=detect_csv
    
    csvs[kind].extend([all_boxes[m][i] for m in all_boxes for i in all_boxes[m]])
    detect_csvs[kind].extend([detect_boxes[m][i] for m in all_boxes for i in all_boxes[m]])

for kind in ['train','val']:
    save_csv_path=os.path.join(root_dir,'AVA','annotations','ava_'+kind+'_v2.2.csv')
    save_csv=open(save_csv_path,'w')
    save_csv.write('\n'.join(csvs[kind]))
    save_csv.close()

    save_detect_csv_path=os.path.join(root_dir,'AVA','annotations','ava_detection_'+kind+'_v2.2.csv')
    save_detect_csv=open(save_detect_csv_path,'w')
    save_detect_csv.write('\n'.join(detect_csvs[kind]))
    save_detect_csv.close()

    save_frame_list_path=os.path.join(root_dir,'AVA','frame_lists',kind+'.csv')
    save_frame_list=open(save_frame_list_path,'w')
    save_frame_list.write('\n'.join(frame_lists[kind]))
    save_frame_list.close()

del csvs
del bad_csvs
del frame_lists


Working on PIGS021219a_000545.mp4
time now = 2024-12-05 15:51:49
time_taken = 0.03 s
Working on 2019-11-11--11:34:50_000000.mp4
time now = 2024-12-05 15:51:53
time_taken = 4.2 s


In [ ]:
for empty_filename in ['ava_train_excluded_timestamps_v2.2.csv','ava_val_excluded_timestamps_v2.2.csv']:
    empty_file=open(os.path.join(root_dir,'AVA','annotations',empty_filename),'w')
    empty_file.close()